In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib

In [ ]:
# Cargar el modelo guardado
modelo = joblib.load(r"air_quality_model\modelo_hibrido_new_0.9269.pkl")

# Cargar los datos originales para obtener todas las estaciones y distritos
df_original = pd.read_csv(r'data/data_monitoreo_contaminantes_filtrado_best.csv')

# Datos históricos de PM10
historico_pm10 = {
    2015: 68.51, 2016: 71.85, 2017: 73.25, 2018: 69.36, 
    2019: 56.26, 2020: 53.22, 2021: 60.80, 2022: 56.51,
    2023: 41.89, 2024: 46.72
}

# Datos de arborización SERPAR
arboles_serpar = {
    2020: 82637,
    2021: 160932,
    2022: 150136,
    2023: 86957,
    2024: 229392
}

# Constantes globales
TASA_SUPERVIVENCIA = 0.7
ARBOLES_PROMEDIO = np.mean(list(arboles_serpar.values()))
VOLUMEN_AIRE = 2819000000 * 10  # área de Lima * altura promedio de mezcla

# Nuevos parámetros para niveles objetivo de PM10
NIVEL_OBJETIVO_MIN = 27  # nivel mínimo objetivo
NIVEL_OBJETIVO_MAX = 40.5  # nivel máximo objetivo (quartil superior)

In [15]:
### 3. Preparar escenarios
def crear_datos_prediccion(start_year=2025, end_year=2035):
    """
    Crea DataFrame con todas las combinaciones necesarias para la predicción
    """
    # Obtener todas las estaciones y distritos únicos
    estaciones = df_original['ESTACION'].unique()
    distritos = df_original['DISTRITO'].unique()
    
    datos = []
    for año in range(start_year, end_year + 1):
        for mes in range(1, 13):
            for estacion, distrito in zip(estaciones, distritos):
                for hora in range(24):
                    dia_semana = pd.Timestamp(year=año, month=mes, day=1).weekday()
                    es_fin_semana = 1 if dia_semana >= 5 else 0
                    datos.append({
                        'ESTACION': estacion,
                        'DISTRITO': distrito,
                        'HORA': hora,
                        'DIA_SEMANA': dia_semana,
                        'MES': mes,
                        'AÑO': año,
                        'ES_FIN_DE_SEMANA': es_fin_semana
                    })
    return pd.DataFrame(datos)

def calcular_reduccion_por_arbol():
    """
    Calcula la reducción de PM10 por árbol en μg/m³
    """
    # Reducción anual por árbol en gramos
    reduccion_anual_gramos = 5217.4
    # Convertir a reducción diaria
    reduccion_diaria_gramos = reduccion_anual_gramos / 365
    # Convertir a reducción de concentración en μg/m³
    reduccion_concentracion = (reduccion_diaria_gramos * 1000) / VOLUMEN_AIRE
    
    return reduccion_concentracion

def simular_escenarios(df_pred, arboles_anuales=ARBOLES_PROMEDIO, start_year=2025):
    """
    Simula escenarios con una cantidad específica de árboles plantados anualmente
    """
    prediccion_base = modelo.predict(df_pred)
    prediccion_con_arboles = prediccion_base.copy()
    
    # Calcular reducción por árbol
    reduccion_por_arbol = calcular_reduccion_por_arbol()
    
    años_unicos = df_pred['AÑO'].unique()
    arboles_acumulados = {}
    
    for i, año in enumerate(años_unicos):
        # Árboles acumulados considerando supervivencia
        n_arboles = int(arboles_anuales * TASA_SUPERVIVENCIA * (i + 1))
        arboles_acumulados[año] = n_arboles
        
        # Reducción total por todos los árboles acumulados
        reduccion_concentracion = reduccion_por_arbol * n_arboles
        
        mask_año = df_pred['AÑO'] == año
        prediccion_con_arboles[mask_año] = np.maximum(0, 
            prediccion_con_arboles[mask_año] - reduccion_concentracion)
    
    return prediccion_base, prediccion_con_arboles, arboles_acumulados

def calcular_arboles_necesarios(df_pred, nivel_objetivo, año_objetivo=2035, max_iteraciones=100):
    """
    Calcula cuántos árboles anuales se necesitan para alcanzar un nivel objetivo de PM10
    """
    # Punto de partida: el promedio actual de SERPAR
    arboles_min = ARBOLES_PROMEDIO
    arboles_max = ARBOLES_PROMEDIO * 10  # un límite superior inicial
    
    # Si con el máximo inicial no alcanzamos el objetivo, aumentamos el límite
    _, pred_arboles_max, _ = simular_escenarios(df_pred, arboles_max)
    mask_año_objetivo = df_pred['AÑO'] == año_objetivo
    pm10_con_max_arboles = np.mean(pred_arboles_max[mask_año_objetivo])
    
    # Si aún con el máximo no alcanzamos el objetivo, aumentamos más
    iteracion = 0
    while pm10_con_max_arboles > nivel_objetivo and iteracion < 5:
        arboles_max *= 5
        _, pred_arboles_max, _ = simular_escenarios(df_pred, arboles_max)
        pm10_con_max_arboles = np.mean(pred_arboles_max[mask_año_objetivo])
        iteracion += 1
    
    # Búsqueda binaria para encontrar el número óptimo de árboles
    for _ in range(max_iteraciones):
        arboles_med = (arboles_min + arboles_max) / 2
        _, pred_arboles_med, _ = simular_escenarios(df_pred, arboles_med)
        mask_año_objetivo = df_pred['AÑO'] == año_objetivo
        pm10_con_arboles_med = np.mean(pred_arboles_med[mask_año_objetivo])
        
        if abs(pm10_con_arboles_med - nivel_objetivo) < 0.1:
            # Convergencia encontrada
            return int(arboles_med)
        
        if pm10_con_arboles_med > nivel_objetivo:
            # Necesitamos más árboles
            arboles_min = arboles_med
        else:
            # Podemos reducir la cantidad de árboles
            arboles_max = arboles_med
    
    # Devolvemos la mejor aproximación encontrada
    return int((arboles_min + arboles_max) / 2)

def categorizar_calidad_aire(valor):
    if valor <= 54:
        return 'Buena'
    elif valor <= 154:
        return 'Moderada'
    elif valor <= 254:
        return 'Insalubre para grupos sensibles'
    else:
        return 'Insalubre'

In [16]:
# 4. Crear datos para predicción
df_prediccion = crear_datos_prediccion(2025, 2035)

# Calcular árboles necesarios para niveles objetivo
arboles_para_nivel_max = calcular_arboles_necesarios(df_prediccion, NIVEL_OBJETIVO_MAX)
arboles_para_nivel_min = calcular_arboles_necesarios(df_prediccion, NIVEL_OBJETIVO_MIN)

# Simular con diferentes cantidades de árboles
pred_base, pred_arboles_actual, arboles_actual = simular_escenarios(df_prediccion, ARBOLES_PROMEDIO)
_, pred_arboles_objetivo_max, arboles_objetivo_max = simular_escenarios(df_prediccion, arboles_para_nivel_max)
_, pred_arboles_objetivo_min, arboles_objetivo_min = simular_escenarios(df_prediccion, arboles_para_nivel_min)

# Crear visualizaciones
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=('Evolución histórica y proyección del PM10',
                                  'Árboles acumulados según diferentes escenarios'))

# Añadir datos históricos
fig.add_trace(
    go.Scatter(x=list(historico_pm10.keys()),
               y=list(historico_pm10.values()),
               name='Histórico PM10',
               line=dict(color='gray', dash='dash')),
    row=1, col=1
)

# Procesar resultados por año con margen de error
margen_error = 0.05  # 5% de margen de error

In [17]:
#5. Función para procesar resultados
def procesar_resultados(pred_base, pred_arboles, arboles_acum, etiqueta):
    resultados = []
    for año in range(2025, 2036):
        mask_año = df_prediccion['AÑO'] == año
        pm10_sin_arboles = np.mean(pred_base[mask_año])
        pm10_con_arboles = np.mean(pred_arboles[mask_año])
        
        # Calcular márgenes de error
        error_sin_arboles = pm10_sin_arboles * margen_error
        error_con_arboles = pm10_con_arboles * margen_error
        
        resultados.append({
            'Año': año,
            'PM10_Sin_Arboles': pm10_sin_arboles,
            'PM10_Sin_Arboles_Min': pm10_sin_arboles - error_sin_arboles,
            'PM10_Sin_Arboles_Max': pm10_sin_arboles + error_sin_arboles,
            'PM10_Con_Arboles': pm10_con_arboles,
            'PM10_Con_Arboles_Min': pm10_con_arboles - error_con_arboles,
            'PM10_Con_Arboles_Max': pm10_con_arboles + error_con_arboles,
            'Reduccion': pm10_sin_arboles - pm10_con_arboles,
            'Arboles_Acumulados': arboles_acum[año],
            'Etiqueta': etiqueta
        })
    return pd.DataFrame(resultados)

# Procesar resultados para cada escenario
resultados_actual = procesar_resultados(pred_base, pred_arboles_actual, arboles_actual, 'Actual')
resultados_objetivo_max = procesar_resultados(pred_base, pred_arboles_objetivo_max, arboles_objetivo_max, f'Para nivel {NIVEL_OBJETIVO_MAX}')
resultados_objetivo_min = procesar_resultados(pred_base, pred_arboles_objetivo_min, arboles_objetivo_min, f'Para nivel {NIVEL_OBJETIVO_MIN}')

# Combinar resultados
resultados_df = pd.concat([resultados_actual, resultados_objetivo_max, resultados_objetivo_min])

# Añadir líneas de predicción sin árboles
fig.add_trace(
    go.Scatter(x=resultados_actual['Año'],
               y=resultados_actual['PM10_Sin_Arboles'],
               name='Proyección sin árboles',
               line=dict(color='red')),
    row=1, col=1
)

# Añadir banda de error para proyección sin árboles
fig.add_trace(
    go.Scatter(
        x=resultados_actual['Año'].tolist() + resultados_actual['Año'].tolist()[::-1],
        y=resultados_actual['PM10_Sin_Arboles_Max'].tolist() + resultados_actual['PM10_Sin_Arboles_Min'].tolist()[::-1],
        fill='toself',
        fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='rgba(255,0,0,0)'),
        name='Margen de error sin árboles',
        showlegend=True
    ),
    row=1, col=1
)

# Añadir líneas para cada escenario
colores = {
    'Actual': 'green',
    f'Para nivel {NIVEL_OBJETIVO_MAX}': 'blue',
    f'Para nivel {NIVEL_OBJETIVO_MIN}': 'purple'
}

for etiqueta, color in colores.items():
    datos = resultados_df[resultados_df['Etiqueta'] == etiqueta]
    fig.add_trace(
        go.Scatter(x=datos['Año'],
                y=datos['PM10_Con_Arboles'],
                name=f'PM10 con árboles ({etiqueta})',
                line=dict(color=color)),
        row=1, col=1
    )
    
    # Añadir árboles acumulados en el segundo gráfico
    fig.add_trace(
        go.Bar(x=datos['Año'],
            y=datos['Arboles_Acumulados'],
            name=f'Árboles acumulados ({etiqueta})',
            marker_color=color),
        row=2, col=1
    )

# Añadir líneas de referencia para niveles objetivo
fig.add_hline(y=NIVEL_OBJETIVO_MAX, line_dash="dot", line_color="blue",
              annotation_text=f"Nivel objetivo max: {NIVEL_OBJETIVO_MAX}", row=1, col=1)
fig.add_hline(y=NIVEL_OBJETIVO_MIN, line_dash="dot", line_color="purple",
              annotation_text=f"Nivel objetivo min: {NIVEL_OBJETIVO_MIN}", row=1, col=1)

# Añadir líneas de referencia para categorías de calidad del aire
categorias = {
    'Buena': 54,
    'Moderada': 154
}

for categoria, valor in categorias.items():
    fig.add_hline(y=valor, line_dash="dash", line_color="gray",
                  annotation_text=categoria, row=1, col=1)

# Actualizar diseño
fig.update_layout(
    title='Simulación de PM10 en Lima Metropolitana: Árboles Necesarios para Niveles Meta',
    height=800,
    showlegend=True,
    xaxis=dict(rangeslider=dict(visible=True))
)

fig.show()

In [18]:
#6. Imprimir estadísticas con margen de error
print("\nEstadísticas de la simulación:")
print(f"\nPromedio actual de árboles plantados por año (SERPAR): {int(ARBOLES_PROMEDIO):,}")
print(f"Árboles anuales necesarios para nivel PM10 de {NIVEL_OBJETIVO_MAX} µg/m³: {arboles_para_nivel_max:,}")
print(f"Árboles anuales necesarios para nivel PM10 de {NIVEL_OBJETIVO_MIN} µg/m³: {arboles_para_nivel_min:,}")
print(f"Tasa de supervivencia considerada: {TASA_SUPERVIVENCIA*100}%")
print(f"Margen de error considerado: ±{margen_error*100}%")

# Calcular la reducción por árbol para referencia
reduccion_por_arbol = calcular_reduccion_por_arbol()
print(f"\nReducción de concentración por árbol: {reduccion_por_arbol:.8f} µg/m³")

# Imprimir resultados por escenario
for etiqueta, resultados in [('Escenario actual', resultados_actual), 
                            (f'Escenario para nivel {NIVEL_OBJETIVO_MAX}', resultados_objetivo_max),
                            (f'Escenario para nivel {NIVEL_OBJETIVO_MIN}', resultados_objetivo_min)]:
    print(f"\n=== {etiqueta} ===")
    for _, row in resultados.iterrows():
        print(f"\nAño {int(row['Año'])}:")
        print(f"Árboles acumulados: {int(row['Arboles_Acumulados']):,}")
        print(f"Reducción promedio: {row['Reduccion']:.2f} µg/m³")
        print(f"PM10 sin árboles: {row['PM10_Sin_Arboles']:.2f} µg/m³ (±{row['PM10_Sin_Arboles']*margen_error:.2f})")
        print(f"PM10 con árboles: {row['PM10_Con_Arboles']:.2f} µg/m³ (±{row['PM10_Con_Arboles']*margen_error:.2f})")


Estadísticas de la simulación:

Promedio actual de árboles plantados por año (SERPAR): 142,010
Árboles anuales necesarios para nivel PM10 de 40.5 µg/m³: 1,936,006
Árboles anuales necesarios para nivel PM10 de 27 µg/m³: 5,415,271
Tasa de supervivencia considerada: 70.0%
Margen de error considerado: ±5.0%

Reducción de concentración por árbol: 0.00000051 µg/m³

=== Escenario actual ===

Año 2025:
Árboles acumulados: 99,407
Reducción promedio: 0.05 µg/m³
PM10 sin árboles: 47.65 µg/m³ (±2.38)
PM10 con árboles: 47.60 µg/m³ (±2.38)

Año 2026:
Árboles acumulados: 198,815
Reducción promedio: 0.10 µg/m³
PM10 sin árboles: 47.40 µg/m³ (±2.37)
PM10 con árboles: 47.29 µg/m³ (±2.36)

Año 2027:
Árboles acumulados: 298,222
Reducción promedio: 0.15 µg/m³
PM10 sin árboles: 47.33 µg/m³ (±2.37)
PM10 con árboles: 47.18 µg/m³ (±2.36)

Año 2028:
Árboles acumulados: 397,630
Reducción promedio: 0.20 µg/m³
PM10 sin árboles: 48.26 µg/m³ (±2.41)
PM10 con árboles: 48.06 µg/m³ (±2.40)

Año 2029:
Árboles acumulados